<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/Web_image_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**画像のスクレイピング**

準備：
- Microsoft AZUREに登録

    https://learn.microsoft.com/ja-jp/azure/cognitive-services/bing-web-search/

- 左のタブ → リソースの作成 → Bing Search v7を取得

- ダッシュボード → キーとエンドポイントからキーを取得する


In [1]:
from google.colab import drive
drive.mount("/content/drive")

with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

bing_api_key = key[13]

Mounted at /content/drive


In [ ]:
import csv
import os

from requests import exceptions
import argparse
import requests
import cv2


API_KEY = f"{bing_api_key}"
MAX_RESULTS = 300
GROUP_SIZE = 1 #GROUP_SIZE×5が一度にリクエストされる

# 取得したエンドポイントURL
URL = "https://api.bing.microsoft.com/v7.0/images/search"
OUTPUT = '/content/drive/MyDrive/Deep_learning/CorneAI_osaka'

if not os.path.isdir(OUTPUT):
    os.mkdir(OUTPUT)

EXCEPTIONS = set([IOError, FileNotFoundError,
                  exceptions.RequestException, exceptions.HTTPError,
                  exceptions.ConnectionError, exceptions.Timeout])

search_terms = ["corneal infection", "healthy eye", "autoimmune keratitis", "corneal scar", "corneal conjunctival tumor",
                "corneal deposit", "angular closure glaucoma photo", "cataract", "bullous keratopathy"]

# set the output csv file name
csv_file = "url_list.csv"

# create the csv file and write the headers
with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Search term', 'Image number', 'Image URL', 'Website URL'])

# loop over each search term and download images
for term in search_terms:
    print(f"[INFO] searching Bing API for '{term}'")

    # create the directory to save the images for the current search term
    output_dir = os.path.join(OUTPUT, term)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    headers = {"Ocp-Apim-Subscription-Key": API_KEY}
    params = {"q": term, "offset": 0, "count": GROUP_SIZE, "imageType": "Photo", "color": "ColorOnly"}

    # make the search
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()

    # grab the results from the search, including the total number of
    # estimated results returned by the Bing API
    results = search.json()
    est_num_results = min(results["totalEstimatedMatches"], MAX_RESULTS)
    print(f"[INFO] {est_num_results} total results for '{term}'")

    # initialize the total number of images downloaded thus far
    total = 0



# loop over each search term and download images
for term in search_terms:
    
    download_urls = [] #画像重複

    print(f"[INFO] searching Bing API for '{term}'")

    # create the directory to save the images for the current search term
    output_dir = os.path.join(OUTPUT, term)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    headers = {"Ocp-Apim-Subscription-Key": API_KEY}
    params = {"q": term, "offset": 0, "count": GROUP_SIZE, "imageType": "Photo", "color": "ColorOnly"}

    # make the search
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()

    # grab the results from the search, including the total number of
    # estimated results returned by the Bing API
    results = search.json()
    est_num_results = min(results["totalEstimatedMatches"], MAX_RESULTS)
    print(f"[INFO] {est_num_results} total results for '{term}'")

    # Determine an appropriate GROUP_SIZE based on the estimated number of results
    GROUP_SIZE = est_num_results//5 # In this example, we divide the estimated number of results by 5
    if GROUP_SIZE == 0:
        GROUP_SIZE = 1 # Make sure that GROUP_SIZE is at least 1
    print(f"[INFO] GROUP_SIZE for '{term}': {GROUP_SIZE}")

    # loop over the estimated number of results in `GROUP_SIZE` groups
    total = 0
    for offset in range(0, est_num_results, GROUP_SIZE):
        # update the search parameters using the current offset, then
        # make the request to fetch the results
        params["offset"] = offset
        params["count"] = GROUP_SIZE # Update the count parameter with the appropriate GROUP_SIZE
        search = requests.get(URL, headers=headers, params=params)
        search.raise_for_status()
        results = search.json()

        # loop over the results
        for v in results["value"]:
            # try to download the image
            try:
                # make a request to download the image
                print("[INFO] fetching: {}".format(v["contentUrl"]))

                # 重複するURLはダウンロードしない
                #print(f"download_urls: {download_urls}")
                #print(f"contentsURL: {v['contentUrl']}")
                if v["contentUrl"] in download_urls:
                    print("duplicated request")
                    continue

                r = requests.get(v["contentUrl"], timeout=30)

                download_urls.append(v["contentUrl"])


                # build the path to the output image
                ext = v["contentUrl"][v["contentUrl"].rfind("."):]
                filename = f"{term}_{str(total).zfill(3)}{ext}"
                output_path = os.path.join(output_dir, filename)

                # write the image to disk
                with open(output_path, "wb") as f:
                    f.write(r.content)

                # write the URL to the csv file
                with open(f"{OUTPUT}/{csv_file}", 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([term, str(total), v["contentUrl"], v["hostPageUrl"]])

                # increment the total number of images downloaded
                total += 1

                # break out of the loop if we have downloaded the maximum number of images
                if total == MAX_RESULTS:
                    break

            # catch any errors that would not unable us to download the
            # image
            except Exception as e:
                print(f"[INFO] skipping: {v['contentUrl']}")



In [4]:
import csv
import os
import requests
from requests import exceptions

API_KEY = f"{bing_api_key}"
MAX_RESULTS = 500
GROUP_SIZE = 5
URL = "https://api.bing.microsoft.com/v7.0/images/search"
OUTPUT = '/content/drive/MyDrive/Deep_learning/CorneAI_osaka'

if not os.path.isdir(OUTPUT):
    os.mkdir(OUTPUT)

EXCEPTIONS = set([IOError, FileNotFoundError,
    exceptions.RequestException, exceptions.HTTPError,
    exceptions.ConnectionError, exceptions.Timeout])

search_terms = ["corneal infection", "healthy eye", "autoimmune keratitis", "corneal scar", "corneal conjunctival tumor", "corneal deposit", "angular closure glaucoma photo", "cataract", "bullous keratopathy"]

csv_file = "url_list.csv"

with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Search term', 'Image number', 'Image URL', 'Website URL'])

for term in search_terms:
    print(f"[INFO] searching Bing API for '{term}'")
    output_dir = os.path.join(OUTPUT, term)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    headers = {"Ocp-Apim-Subscription-Key": API_KEY}
    params = {"q": term, "offset": 0, "count": GROUP_SIZE, "imageType": "Photo", "color": "ColorOnly"}

    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()

    results = search.json()
    est_num_results = min(results["totalEstimatedMatches"], MAX_RESULTS)
    print(f"[INFO] {est_num_results} total results for '{term}'")

    total = 0
    for offset in range(0, est_num_results, GROUP_SIZE):
        params["offset"] = offset
        search = requests.get(URL, headers=headers, params=params)
        search.raise_for_status()
        results = search.json()
        
        for v in results["value"]:
            try:
                print("[INFO] fetching: {}".format(v["contentUrl"]))
                r = requests.get(v["contentUrl"], timeout=30)

                ext = v["contentUrl"][v["contentUrl"].rfind("."):]
                filename = f"{term}_{str(total).zfill(3)}{ext}"
                output_path = os.path.join(output_dir, filename)

                with open(output_path, "wb") as f:
                    f.write(r.content)

                with open(f"{OUTPUT}/{csv_file}", 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([term, filename, v["contentUrl"], v["hostPageUrl"]])

            except Exception as e:
                print(f"[INFO] skipping: {v['contentUrl']}")

            total += 1
            print(f"{total} images downloaded!")
            if total >= MAX_RESULTS:
                break

        if total >= MAX_RESULTS:
            break


[INFO] searching Bing API for 'corneal infection'
[INFO] 352 total results for 'corneal infection'
[INFO] fetching: https://elza2021-to47xpx0nsmkpskkap.netdna-ssl.com/wp-content/uploads/2020/09/Keratitis.jpg
[INFO] skipping: https://elza2021-to47xpx0nsmkpskkap.netdna-ssl.com/wp-content/uploads/2020/09/Keratitis.jpg
1 images downloaded!
[INFO] fetching: https://i0.wp.com/entokey.com/wp-content/uploads/2016/11/9781451180398_fig7-21f_FIG.jpg?fit=384%2C256&ssl=1
[INFO] skipping: https://i0.wp.com/entokey.com/wp-content/uploads/2016/11/9781451180398_fig7-21f_FIG.jpg?fit=384%2C256&ssl=1
2 images downloaded!
[INFO] fetching: https://thenjeye.com/wp-content/uploads/2019/03/corneal-infections.jpg
[INFO] skipping: https://thenjeye.com/wp-content/uploads/2019/03/corneal-infections.jpg
3 images downloaded!
[INFO] fetching: https://media.sciencephoto.com/image/m1550509/400wm/M1550509-Corneal_infection.jpg
[INFO] skipping: https://media.sciencephoto.com/image/m1550509/400wm/M1550509-Corneal_infectio

KeyboardInterrupt: ignored

#**Chromedriverを用いる方法**

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium==4.1.0 #新しいバージョンだとエラーが出るので旧バージョンにする

In [ ]:
# これだとサムネイルしか取得できない

import requests
from bs4 import BeautifulSoup
import os

# Search query
search_query = "flowers"

# Number of images to download
num_images = 10

# Create a new folder for the images
if not os.path.exists(search_query):
    os.makedirs(search_query)

# URL to search Google Images
url = f"https://www.google.com/search?q={search_query}&tbm=isch"

# Send GET request
response = requests.get(url)

# Parse the HTML using Beautiful Soup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all image tags
images = soup.find_all('img')

# Iterate through the images and download them
for i, img in enumerate(images[:num_images]):
    url = img['src']
    print(i)
    try:
        response = requests.get(url)
        open(f"{search_query}/{search_query}_{i}.jpg", "wb").write(response.content)
    except:
        print("download error")

In [ ]:
!apt-get update
!curl -O https://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_linux64.zip #Chromeのバージョンに合ったchromedriverのアドレスを設定
!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv chromedriver /usr/local/bin/
!pip install selenium

from selenium import webdriver

# Chromeドライバーの設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--disable-browser-side-navigation')

# Googleで検索する
search_query = 'flowers'
url = f"https://www.google.com/search?q={search_query}&tbm=isch"
browser = webdriver.Chrome('chromedriver',options=options)
browser.get(url)


import os
from bs4 import BeautifulSoup
import base64
import requests
from io import BytesIO
from PIL import Image


# 画像のURLを取得する
soup = BeautifulSoup(browser.page_source, 'html.parser')
img_tags = soup.find_all('img', class_='rg_i')


urls = []
for img in img_tags:
    try:
        urls.append(img["src"])
    except:
        pass


# 画像をダウンロードする
if not os.path.exists(search_query):
    os.makedirs(search_query)

num_images = 10

counter = 0
for i in range(num_images):
    print(urls[i])
    image_data = base64.b64decode(urls[i].split(',')[1])

    # バイナリデータをBytesIOオブジェクトに書き込む
    image_stream = BytesIO(image_data)

    # PILで画像オブジェクトを作成する
    image = Image.open(image_stream)
    image_format = image.format

    # 画像のネーミング
    num= "{:04d}".format(i)
    file_name = f"{search_query}_{num}"
    new_image_path = f"{search_query}/{file_name}.{image_format}"


    # Save image to file
    image.save(new_image_path)
